In [4]:
#!/usr/bin/env python3
import sys
import os
import argparse
import math
import time
from scipy.signal import butter, lfilter
from scipy.stats import gamma  # Import gamma for entropy calculation
import fastdyn_fic_dmf as dmf
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from helper_functions import filter_bold
from multiprocessing import Pool

def calculate_gamma_entropy(node_index, rates):
    print(f"Processing node {node_index}")
    # Create arrays to store the results
    alpha = np.zeros((rates.shape[0],))
    loc = np.zeros((rates.shape[0],))
    beta = np.zeros((rates.shape[0],))
    entropy_value = np.zeros((rates.shape[0],))
    # Average over the axis 0 (repetitions) the entropy value
    for i in range(rates.shape[0]):
        print(f"Processing repetition {i}")
        rep_rates = rates[i]
        # Fit a gamma distribution to the data
        alpha[i], loc[i], beta[i] = gamma.fit(rep_rates)
        # Calculate the entropy of the gamma distribution
        entropy_value[i] = gamma.entropy(a=alpha[i], loc=loc[i], scale=beta[i])
    # Average over the repetitions
    alpha_mean = np.mean(alpha)
    loc_mean = np.mean(loc)
    beta_mean = np.mean(beta)
    entropy_mean = np.mean(entropy_value)    
    return node_index, alpha_mean, loc_mean, beta_mean, entropy_mean

def compute_fcd(data, wsize, overlap, isubdiag, params):
    T, N = data.shape
    win_start = np.arange(0, T - params["wsize"] - 1, params["wsize"] - overlap)
    nwins = len(win_start)
    fcd = np.zeros((len(isubdiag[0]), nwins))
    for i in range(nwins):
        tmp = data[win_start[i]:win_start[i] + params["wsize"] + 1, :]
        cormat = np.corrcoef(tmp.T)
        fcd[:, i] = cormat[isubdiag[0], isubdiag[1]]
    return fcd

def grid_step(args):
    """
    Processes a single SEED and returns the results, including entropy per region.
    """
    SEED_tuple, params, nb_steps, burnout, overlap, isubfcd, a, b = args
    idx_SEED, SEED = SEED_tuple
    params['seed'] = SEED
    #print(f"Processing Seed {SEED} (Index {idx_SEED})")
    
    # Initialize a dictionary to store entropy per region
    entropy_per_region = {}
    
    # Statistical FC
    params['G'] = loadmat('../matlab/Results/stat_fc/results_awake_stat_fc.mat')['minEstimatedG_Awake']
    params['alpha'] = loadmat('../matlab/Results/stat_fc/results_awake_stat_fc.mat')['minEstimatedY_Awake']
    params['J'] = params['alpha'] * params['G'] * params['C'].sum(axis=0).squeeze() + 1
    params["with_plasticity"] = False
    params["with_decay"] = False
    rates, rates_inh, bold, fic_t = dmf.run(params, nb_steps)
    bold = bold[:, burnout:]
    filt_bold = filter_bold(bold.T, params['flp'], params['fhp'], params['TR'])
    stat_bold_fc = np.corrcoef(filt_bold.T)
    
    # Calculate entropy for Statistical FC
    entropy_stat_fc = []
    for node in range(params['N']):
        node_rates = rates[:, node]
        _, _, _, _, entropy = calculate_gamma_entropy(node, node_rates.reshape(rates.shape[0], -1))
        entropy_stat_fc.append(entropy)
    entropy_per_region['stat_fc'] = np.array(entropy_stat_fc)
    
    # Statistical FCD
    params['G'] = loadmat('../matlab/Results/stat_fcd/results_awake_stat_fcd.mat')['minEstimatedG_Awake']
    params['alpha'] = loadmat('../matlab/Results/stat_fcd/results_awake_stat_fcd.mat')['minEstimatedY_Awake']
    params['J'] = params['alpha'] * params['G'] * params['C'].sum(axis=0).squeeze() + 1
    rates, rates_inh, bold, fic_t = dmf.run(params, nb_steps)
    bold = bold[:, burnout:]
    filt_bold = filter_bold(bold.T, params['flp'], params['fhp'], params['TR'])
    stat_fcd = compute_fcd(filt_bold, params["wsize"], overlap, isubfcd, params)
    stat_fcd = np.corrcoef(stat_fcd.T)
    
    # Calculate entropy for Statistical FCD
    entropy_stat_fcd = []
    for node in range(params['N']):
        node_rates = rates[:, node]
        _, _, _, _, entropy = calculate_gamma_entropy(node, node_rates.reshape(rates.shape[0], -1))
        entropy_stat_fcd.append(entropy)
    entropy_per_region['stat_fcd'] = np.array(entropy_stat_fcd)
    
    # Dynamic FC
    params['G'] = loadmat('../matlab/Results/dyn_fc/results_awake_dyn_fc.mat')['minEstimatedG_Awake']
    params['lrj'] = loadmat('../matlab/Results/dyn_fc/results_awake_dyn_fc.mat')['minEstimatedY_Awake']
    DECAY = np.exp(a + np.log(params['lrj']) * b)
    params['taoj'] = DECAY
    params['alpha'] = 0.75
    params["with_plasticity"] = True
    params["with_decay"] = True
    params['J'] = params['alpha'] * params['G'] * params['C'].sum(axis=0).squeeze() + 1
    rates, rates_inh, bold, fic_t = dmf.run(params, nb_steps)
    bold = bold[:, burnout:]
    filt_bold = filter_bold(bold.T, params['flp'], params['fhp'], params['TR'])
    dyn_bold_fc = np.corrcoef(filt_bold.T)
    
    # Calculate entropy for Dynamic FC
    entropy_dyn_fc = []
    for node in range(params['N']):
        node_rates = rates[:, node]
        _, _, _, _, entropy = calculate_gamma_entropy(node, node_rates.reshape(rates.shape[0], -1))
        entropy_dyn_fc.append(entropy)
    entropy_per_region['dyn_fc'] = np.array(entropy_dyn_fc)
    
    # Dynamic FCD
    params['G'] = loadmat('../matlab/Results/dyn_fcd/results_awake_dyn_fcd.mat')['minEstimatedG_Awake']
    params['lrj'] = loadmat('../matlab/Results/dyn_fcd/results_awake_dyn_fcd.mat')['minEstimatedY_Awake']
    DECAY = np.exp(a + np.log(params['lrj']) * b)
    params['taoj'] = DECAY
    params['alpha'] = 0.75
    params["with_plasticity"] = True
    params["with_decay"] = True
    params['J'] = params['alpha'] * params['G'] * params['C'].sum(axis=0).squeeze() + 1
    rates, rates_inh, bold, fic_t = dmf.run(params, nb_steps)
    bold = bold[:, burnout:]
    filt_bold = filter_bold(bold.T, params['flp'], params['fhp'], params['TR'])
    dyn_fcd = compute_fcd(filt_bold, params["wsize"], overlap, isubfcd, params)
    dyn_fcd = np.corrcoef(dyn_fcd.T)
    
    # Calculate entropy for Dynamic FCD
    entropy_dyn_fcd = []
    for node in range(params['N']):
        node_rates = rates[:, node]
        _, _, _, _, entropy = calculate_gamma_entropy(node, node_rates.reshape(rates.shape[0], -1))
        entropy_dyn_fcd.append(entropy)
    entropy_per_region['dyn_fcd'] = np.array(entropy_dyn_fcd)
    
    return {
        'idx_SEED': idx_SEED,
        'stat_bold_fc': stat_bold_fc,
        'stat_fcd': stat_fcd,
        'dyn_bold_fc': dyn_bold_fc,
        'dyn_fcd': dyn_fcd,
        'stat_bold': filt_bold,
        'dyn_bold': filt_bold,
        'entropy_stat_fc': entropy_per_region['stat_fc'],
        'entropy_stat_fcd': entropy_per_region['stat_fcd'],
        'entropy_dyn_fc': entropy_per_region['dyn_fc'],
        'entropy_dyn_fcd': entropy_per_region['dyn_fcd']
    }

def integrate_results(results, N, wsize, bold_length):
    """
    Integrates all results and saves the aggregated data, including entropy per region.
    """
    print("Integrating results...")
    
    SEED_count = len(results)
    mean_fc_grid = np.zeros((2, SEED_count, N, N))
    sim_fcds_grid = np.zeros((2, SEED_count, wsize, wsize))  # Adjust dimensions as needed
    bold_grid = {
        'stat_bold': np.zeros((SEED_count, bold_length, N)),
        'dyn_bold': np.zeros((SEED_count, bold_length, N))
    }
    entropy_grid = {
        'entropy_stat_fc': np.zeros((SEED_count, N)),
        'entropy_stat_fcd': np.zeros((SEED_count, N)),
        'entropy_dyn_fc': np.zeros((SEED_count, N)),
        'entropy_dyn_fcd': np.zeros((SEED_count, N))
    }
    
    for partial in results:
        idx_SEED = partial['idx_SEED']
        stat_bold_fc = partial['stat_bold_fc']
        stat_fcd = partial['stat_fcd']
        dyn_bold_fc = partial['dyn_bold_fc']
        dyn_fcd = partial['dyn_fcd']
        stat_bold = partial['stat_bold']
        dyn_bold = partial['dyn_bold']
        
        entropy_stat_fc = partial['entropy_stat_fc']
        entropy_stat_fcd = partial['entropy_stat_fcd']
        entropy_dyn_fc = partial['entropy_dyn_fc']
        entropy_dyn_fcd = partial['entropy_dyn_fcd']

        mean_fc_grid[0, idx_SEED] = stat_bold_fc
        sim_fcds_grid[0, idx_SEED] = stat_fcd
        mean_fc_grid[1, idx_SEED] = dyn_bold_fc
        sim_fcds_grid[1, idx_SEED] = dyn_fcd
        bold_grid['stat_bold'][idx_SEED] = stat_bold
        bold_grid['dyn_bold'][idx_SEED] = dyn_bold
        
        entropy_grid['entropy_stat_fc'][idx_SEED] = entropy_stat_fc
        entropy_grid['entropy_stat_fcd'][idx_SEED] = entropy_stat_fcd
        entropy_grid['entropy_dyn_fc'][idx_SEED] = entropy_dyn_fc
        entropy_grid['entropy_dyn_fcd'][idx_SEED] = entropy_dyn_fcd

    # Save integrated results
    arrays_to_save = {
        'fcs_grid': mean_fc_grid,
        'fcds_grid': sim_fcds_grid,
        'bold_grid_stat': bold_grid['stat_bold'],
        'bold_grid_dyn': bold_grid['dyn_bold'],
        'entropy_stat_fc': entropy_grid['entropy_stat_fc'],
        'entropy_stat_fcd': entropy_grid['entropy_stat_fcd'],
        'entropy_dyn_fc': entropy_grid['entropy_dyn_fc'],
        'entropy_dyn_fcd': entropy_grid['entropy_dyn_fcd']
    }

    final_results_folder = "./Results/FittedSimulations"
    os.makedirs(final_results_folder, exist_ok=True)

    for array_name, array_data in arrays_to_save.items():
        file_name = os.path.join(final_results_folder, f"{array_name}.npy")
        np.save(file_name, array_data)
        print(f"Saved integrated {array_name} to {file_name}")

# Prepare parameters and data
C = loadmat('./data/DTI_fiber_consensus_HCP.mat')['connectivity'][:200, :200]
C = 0.2 * C / np.max(C)
params = dmf.default_params(C=C)

triu_idx = np.triu_indices(C.shape[1], 1)
params['N'] = C.shape[0]
isubfcd = np.triu_indices(C.shape[1], 1)

# Main setup for this simulation
params["return_rate"] = True
params["return_bold"] = True
params["return_fic"] = True

burnout = 7
params["flp"] = 0.01
params["fhp"] = 0.1
params["wsize"] = 30
overlap = 29
params['TR'] = 2

T = 250
params['dtt'] = 0.001  # Assuming 'dtt' is defined; adjust as needed
nb_steps = int(T * params['TR'] / params['dtt'])
win_start = np.arange(0, T - burnout - params["wsize"], params["wsize"] - overlap)
nwins = len(win_start)
nints = len(isubfcd[0])

fit_res = np.load("./data/fit_res_3-44.npy", allow_pickle=True)
b = fit_res[0]  # First element is the slope
a = fit_res[1]

SEED_range = list(range(1, 101))  # SEEDs from 1 to 100

# Adjust the number of cores based on your local machine
# For example, use os.cpu_count() to automatically detect
NUM_CORES = min(24, os.cpu_count())  # Use up to 24 cores or the maximum available

OBJ_RATE = 3.44
params['obj_rate'] = OBJ_RATE

# Prepare all SEEDs for processing
task_seeds = SEED_range
task_indices = list(range(len(task_seeds)))
task_args = [((idx_SEED, SEED), params.copy(), nb_steps, burnout, overlap, isubfcd, a, b)
            for idx_SEED, SEED in zip(task_indices, task_seeds)]

# Process SEEDs using multiprocessing Pool
print(f"Starting processing of {len(task_seeds)} SEEDs using {NUM_CORES} cores...")
with Pool(processes=NUM_CORES) as pool:
    results = pool.map(grid_step, task_args)

#return idx_SEED, mean_fc, sim_fcds,mean_firing_rates, std_firing_rates
for results in results_list:
    idx_SEED = results[0]        
    stat_bold_fc = results[1]
    stat_fcd = results[2] 
    dyn_bold_fc = results[3]  
    dyn_fcd = results[4]
    stat_bold = results[5]
    dyn_bold = results[6]
    
    mean_fc_grid[0,idx_SEED] = stat_bold_fc
    sim_fcds_grid[0,idx_SEED] = stat_fcd
    mean_fc_grid[1,idx_SEED] = dyn_bold_fc
    sim_fcds_grid[1,idx_SEED] = dyn_fcd
    bold_grid[0,idx_SEED] = stat_bold
    bold_grid[1,idx_SEED] = dyn_bold
    


import os

# Assuming these arrays are already populated with data

arrays_to_save = {
    'fcs_grid': mean_fc_grid,
    'fcds_grid': sim_fcds_grid,
    'bold_grid': bold_grid,
}

results_folder = "./Results/FittedSimulations"

# Save
for array_name, array_data in arrays_to_save.items():
    file_name = os.path.join(results_folder, f"{array_name}.npy")
    np.save(file_name, array_data)


In [5]:
import os
import numpy as np
arrays_to_save = ['fcs_grid',
    'fcds_grid',]
    #'bold_grid']

results_folder = "./Results/FittedSimulations"
results = {}
for array_name in arrays_to_save:
    file_name = os.path.join(results_folder, f"{array_name}.npy")
    results[array_name] = np.load(file_name)

In [30]:
results = loadmat('/network/iss/home/ivan.mindlin/dyn_fic_dmf/matlab/Results/dyn_fcd/results_awake_dyn_fcd.mat')
params['G'] = 2.1#results['minEstimatedG_Awake'][0][0]
params['lrj'] = 10#results['minEstimatedY_Awake'][0][0]
DECAY = np.exp(a+np.log(params['lrj'])*b)
params['taoj'] = DECAY 
params['J'] = 0.75 * params['G'] * params['C'].sum(axis=0).squeeze() + 1    
params["with_plasticity"] = True
params["with_decay"] = True  
params['seed'] = 1
# Run the simulation
rates, rates_inh, bold, fic_t = dmf.run(params, nb_steps)         
#bold = bold[:, burnout:]
#filt_bold = filter_bold(bold.T, params['flp'], params['fhp'], params['TR'])

Debuging bold

In [20]:
# Save a pickle with the results of the simulation and the params
import pickle
with open('Results/debug_python_compiled.pkl', 'wb') as f:
    pickle.dump({'rates': rates, 'rates_inh': rates_inh, 'bold': bold, 'params': params}, f)

In [32]:
matlab_test = loadmat('/network/iss/home/ivan.mindlin/dyn_fic_dmf/matlab/Results/debug_matlab_recompiled_nonan.mat')